In [ ]:
!ls 3/*/* --full-time -Ggh

In [2]:
!ls 3/traindata/history_behavior_data/20210419/

part-00000-236b99d5-456a-42b2-bd8d-3cbd61d21cc6-c000.csv


# 数据读取

In [1]:
import pandas as pd
import glob, gc

%pylab inline
import seaborn as sns

INPUT_PATH = './3'

Populating the interactive namespace from numpy and matplotlib


In [3]:
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

test_data = pd.read_csv(f'{INPUT_PATH}/testdata/test.csv', sep=',')
user_features = pd.read_csv(f'{INPUT_PATH}/traindata/user_features_data/user_features_data.csv', sep='\t')
video_features = pd.read_csv(f'{INPUT_PATH}/traindata/video_features_data/video_features_data.csv', sep='\t')
history_behavior = pd.concat([
    reduce_mem(pd.read_csv(x, sep='\t')) for x in glob.glob(f'{INPUT_PATH}/traindata/history_behavior_data/*/*')
])
# 日期和用户id
history_behavior = history_behavior.sort_values(by=['pt_d', 'user_id'])

test_data = reduce_mem(test_data)
user_features = reduce_mem(user_features)
video_features = reduce_mem(video_features)
history_behavior = reduce_mem(history_behavior)

Populating the interactive namespace from numpy and matplotlib


/home/lyz/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


378.43 Mb, 131.40 Mb (65.28 %)
405.14 Mb, 140.67 Mb (65.28 %)
437.46 Mb, 151.90 Mb (65.28 %)
374.51 Mb, 130.04 Mb (65.28 %)
387.82 Mb, 134.66 Mb (65.28 %)
372.54 Mb, 129.35 Mb (65.28 %)
383.36 Mb, 133.11 Mb (65.28 %)
397.30 Mb, 137.95 Mb (65.28 %)
444.39 Mb, 154.30 Mb (65.28 %)
347.10 Mb, 120.52 Mb (65.28 %)
368.09 Mb, 127.81 Mb (65.28 %)
448.31 Mb, 155.66 Mb (65.28 %)
388.03 Mb, 134.73 Mb (65.28 %)
379.69 Mb, 131.84 Mb (65.28 %)
43.06 Mb, 21.53 Mb (50.00 %)
360.77 Mb, 73.28 Mb (79.69 %)
3.79 Mb, 3.04 Mb (20.00 %)
2526.41 Mb, 2526.41 Mb (0.00 %)


In [3]:
test_data.to_hdf('digix-data.hdf', 'test_data')
user_features.to_hdf('digix-data.hdf', 'user_features')
video_features.to_hdf('digix-data.hdf', 'video_features')
history_behavior.to_hdf('digix-data.hdf', 'history_behavior')

NameError: name 'test_data' is not defined

In [2]:
test_data = pd.read_hdf('digix-data.hdf', 'test_data')
user_features = pd.read_hdf('digix-data.hdf', 'user_features')
video_features = pd.read_hdf('digix-data.hdf', 'video_features')
history_behavior = pd.read_hdf('digix-data.hdf', 'history_behavior')

In [ ]:
history_behavior

In [4]:
# history_behavior = history_behavior[history_behavior['user_id'].isin(test_data['user_id'].unique())]
val_behavior = history_behavior[history_behavior['pt_d'] == 20210502]
train_behavior = history_behavior[history_behavior['pt_d'] != 20210502]

In [5]:
train_behavior = pd.concat([
    train_behavior[train_behavior['watch_label'] == 0].sample(1000000),
    train_behavior[train_behavior['watch_label'] != 0],
])

In [6]:
val_behavior = pd.concat([
    val_behavior[val_behavior['watch_label'] == 0].sample(1000000),
    val_behavior[val_behavior['watch_label'] != 0],
])

In [7]:
val_behavior['watch_label'].value_counts()

0    1000000
1      34361
9      27680
2      20217
3      14623
4      11779
5       9748
8       9711
6       8676
7       7986
Name: watch_label, dtype: int64

In [8]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch import nn
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(0)

class MLP(nn.Module):
    # 行为数据
    def __init__(self, n_users=5910798+1, n_items=50355+1, 
                 n_provinces=33+1, n_citys=339+1, n_devices=1826+1,
                 layers=[70, 32], dropout=False):
        super().__init__()
        self.user_embedding = torch.nn.Embedding(n_users, 50)
        self.video_embedding = torch.nn.Embedding(n_items, 20)
        
        # self.user_province_embedding = torch.nn.Embedding(n_provinces, 5)
        # self.user_city_embedding = torch.nn.Embedding(n_citys, 5)
        # self.user_device_embedding = torch.nn.Embedding(n_devices, 5)

        # list of weight matrices
        self.fc_layers = torch.nn.ModuleList()
        for _, (in_size, out_size) in enumerate(zip(layers[:-1], layers[1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
        self.output_layer1 = torch.nn.Linear(layers[-1], 10)
        self.output_layer2 = torch.nn.Linear(layers[-1], 1)
        
        print(self.fc_layers)
        
    def forward(self, feed_dict):
        users = feed_dict['user_id']
        items = feed_dict['video_id']        
        
        user_embedding = self.user_embedding(users)
        video_embedding = self.video_embedding(items)
        
        # user_province = self.user_province_embedding(feed_dict['user_province'])
        # user_city = self.user_city_embedding(feed_dict['user_city'])
        # user_device = self.user_device_embedding(feed_dict['user_device'])
        
        x = torch.cat([user_embedding, video_embedding], 1)
        # x = torch.cat([user_embedding, video_embedding, user_province, user_city, user_device], 1)
        for idx, _ in enumerate(range(len(self.fc_layers))):
            x = self.fc_layers[idx](x)
            x = F.relu(x)
            x = F.dropout(x)
        logit1 = self.output_layer1(x)
        logit2 = self.output_layer2(x)
        return logit1, logit2

    def predict(self, feed_dict):
        for key in feed_dict:
            if type(feed_dict[key]) != type(None):
                feed_dict[key] = torch.from_numpy(
                    feed_dict[key]).to(dtype=torch.long, device='cpu')
        output_scores = self.forward(feed_dict)
        return output_scores

In [9]:
model = MLP()
model.predict({'user_id':np.array([10, 10]), 
               'video_id':np.array([10, 10]),
              'user_city': np.array([10, 10]), 
               'user_province': np.array([10, 10]), 
               'user_device':np.array([10, 10])})

ModuleList(
  (0): Linear(in_features=70, out_features=32, bias=True)
)


(tensor([[-0.0339, -0.1640, -0.1777,  0.3275,  0.1747, -0.1851, -0.0804, -0.1758,
          -0.1822, -0.0843],
         [-0.3471,  0.0056,  0.1633,  0.3907, -0.4149, -0.3281,  0.3300, -0.1325,
          -0.0363,  0.0784]], grad_fn=<AddmmBackward>),
 tensor([[ 0.1139],
         [-0.3359]], grad_fn=<AddmmBackward>))

In [10]:
model = model.cuda()


# 正样本有多少？ 

In [ ]:
class MLPDataset(Dataset):
    def __init__(self, history_behavior, train=True):
        self.history_behavior = history_behavior
        self.train = train
        
    def __getitem__(self, index):
        user_id = self.history_behavior.iloc[index]['user_id']
        video_id = self.history_behavior.iloc[index]['video_id']
        
        # 从行为日志 去 索引用户的特征
        user_province = user_features.loc[user_id]['province']
        user_city = user_features.loc[user_id]['city']
        user_device = user_features.loc[user_id]['device_name']
        
        feed_dict = {
            'user_id': user_id,
            'video_id': video_id,
            'user_province': user_province,
            'user_city': user_city,
            'user_device': user_device,
        }
        
        if self.train:
            watch_label = self.history_behavior.iloc[index]['watch_label']
            share_label = self.history_behavior.iloc[index]['is_share']

            feed_dict['watch_label'] = watch_label
            feed_dict['share_label'] = share_label
            
            return feed_dict
            
#             return user_id, video_id, \
#                 torch.from_numpy(np.array(watch_label)), \
#                 torch.from_numpy(np.array([share_label]))
        else:
            return feed_dict
        
    def __len__(self):
        return len(self.history_behavior)
    
train_loader = torch.utils.data.DataLoader(
    dataset = MLPDataset(train_behavior),
    batch_size=1000, shuffle=True, num_workers=5,
)

# batch_size 1000

val_loader = torch.utils.data.DataLoader(
    dataset = MLPDataset(val_behavior, train=True),
    batch_size=1000, shuffle=False, num_workers=5,
)

In [12]:
train_behavior['watch_label'].value_counts()

0    1000000
1     523060
9     360841
2     293890
3     204565
4     160625
5     133253
8     129087
6     116416
7     109763
Name: watch_label, dtype: int64

In [13]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
writer = SummaryWriter('multi-task-dl')

/home/lyz/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
wathch_loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor([1,2,2,2,2,2,2,2,2,2]).cuda())
shaere_loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

step = 0
for epoch in range(10):
    for idx, data in tqdm(enumerate(train_loader),total=len(train_loader), desc=f'Epoch {epoch}'):
        feed_dict_cuda = {
            'user_id': data['user_id'].long().cuda(),
            'video_id': data['video_id'].long().cuda(),
            'user_province': data['user_province'].long().cuda(),
            'user_city': data['user_city'].long().cuda(),
            'user_device': data['user_device'].long().cuda(),
        }
        
        watch_label = data['watch_label'].long().cuda()
        share_label = data['share_label'].float().cuda().view(-1, 1)

        optimizer.zero_grad()
        wathch_pred, share_pred = model(feed_dict_cuda)
        loss = wathch_loss_fn(wathch_pred, watch_label) # + shaere_loss_fn(share_pred, share_label)

        loss.backward()
        optimizer.step()

        total_acc = (wathch_pred.argmax(1) == watch_label).float().mean().item()
        true_acc = ((wathch_pred.argmax(1) == watch_label) & (watch_label != 0)).float().sum()
        true_acc /= (watch_label != 0).float().sum()
        
        writer.add_scalar('Train/Total-ACC', total_acc, step)
        writer.add_scalar('Train/True-ACC', true_acc, step)
        writer.add_scalar('Train/Loss', loss.item(), step)
        writer.flush()
        step += 1
                
        if idx!=0 and idx % 1000 == 0:
            val_acc = []
            val_pred, val_label = [], []
            with torch.no_grad():
                for data in tqdm(val_loader, desc=f'Val'):
                    val_label += list(data['watch_label'].data.numpy())

                    feed_dict_cuda = {
                        'user_id': data['user_id'].long().cuda(),
                        'video_id': data['video_id'].long().cuda(),
                        'user_province': data['user_province'].long().cuda(),
                        'user_city': data['user_city'].long().cuda(),
                        'user_device': data['user_device'].long().cuda(),
                    }

                    watch_label = data['watch_label'].long().cuda()
                    share_label = data['share_label'].float().cuda().view(-1, 1)

                    wathch_pred, share_pred = model(feed_dict_cuda)
                    
                    val_pred += list(wathch_pred.argmax(1).data.cpu().numpy())
                    true_acc = ((wathch_pred.argmax(1) == watch_label) & (watch_label != 0)).float().sum()
                    true_acc /= (watch_label != 0).float().sum()
                    val_acc.append(true_acc)
            
            val_label = np.array(val_label)
            val_pred = np.array(val_pred)
            from sklearn.metrics import roc_auc_score
            score = 0 
            for aucflag, lbl in zip(np.linspace(0.1, 0.9, 9), range(1, 10)):
                # [0, 1, 2, 3] -> [0, 0, 1, 0]
                # [1, 1, 2, 3] -> [0, 0, 1, 0]
                pred = (val_pred == lbl).astype(int)
                label = (val_label == lbl).astype(int)
                try:
                    score += aucflag * roc_auc_score(label, pred)
                except:
                    pass
            
            val_acc = pd.DataFrame([x.item() for x in val_acc]).fillna(0).mean()[0]
            writer.add_scalar('Val/True-ACC', val_acc, step)
            writer.add_scalar('Val/AUC', score, step)
            writer.flush()

Val:  25%|██▍       | 286/1145 [00:51<02:41,  5.33it/s]

In [78]:
test_loader = torch.utils.data.DataLoader(
    dataset = MLPDataset(test_data, train=False),
    batch_size=20, shuffle=False, num_workers=5,
)

test_watch = []
test_share = []
with torch.no_grad():
    for data in test_loader:
        feed_dict_cuda = {
            'user_id': data['user_id'].long().cuda(),
            'video_id': data['video_id'].long().cuda(),
            'user_province': data['user_province'].long().cuda(),
            'user_city': data['user_city'].long().cuda(),
            'user_device': data['user_device'].long().cuda(),
        }
        wathch_pred, share_pred = model(feed_dict_cuda)
        
        test_watch += list(wathch_pred.argmax(1).cpu().data.numpy())
        test_share += list((share_pred.sigmoid() > 0.5).int().cpu().data.numpy().flatten())

In [79]:
test_data['watch_label'] = test_watch
test_data['is_share'] = test_share

In [80]:
test_data.to_csv('submission.csv', index=None)

In [82]:
!zip submission.csv.zip submission.csv

  adding: submission.csv (deflated 64%)


In [88]:
test_data['is_share'].value_counts()

0    2822180
Name: is_share, dtype: int64

In [89]:
test_data['watch_label'].value_counts()

0    2626811
2      38000
1      36062
9      33947
3      25696
4      19315
5      15689
6      15392
8       6866
7       4402
Name: watch_label, dtype: int64